## Twitter

### Twitter Authentication Information

In [1]:
# This line allows us to access the TwitterAPI library
from TwitterAPI import TwitterAPI
import pprint
import psycopg2
import time
import sys
import jsonconfig as jsc

In [2]:
# Import credentials from config file
twitter_cred = jsc.json_config('config/twitter_config.json')

# Consumer key and secret key
consumer_key = twitter_cred['twitter']['consumerAPIKey']
consumer_secret = twitter_cred['twitter']['consumerSecretKey']

# Access key and secret key
access_token_key = twitter_cred['twitter']['accessToken']
access_token_secret = twitter_cred['twitter']['accessTokenSecret']

# Test Connection
try:
    api = TwitterAPI(consumer_key, consumer_secret, access_token_key, access_token_secret,)
    print("Consumer and access keys and secrets have been successfully added")
except:
    print("Something went wrong with adding the credentials")

Consumer and access keys and secrets have been successfully added


### Connect and Create a Cursor for a Postgres Database

Instead of writing the data collected from Twitter to a csv file, I chose to write the streamed data to a postgres database using the psycopg2 module in Python. This way I do not have to worry about any cocurrency issues when I query data from the database if data is still being writing to it. This also allows multiple users/scripts to write to the database which could be useful with multiple people working on the same project. The database is locally hosted (At the moment), so passwords and other information are close to default. If the database was hosted somewhere, I would explore the security measures that need to be in place. 

In [3]:
# A database needs to be created already and user/password should be configured as well

# Import credentials from config file
psql_cred = jsc.json_config('config/psql_config.json')
dbname = psql_cred['psql']['dbname']
host = psql_cred['psql']['host']
port = psql_cred['psql']['port']
user = psql_cred['psql']['user']
password = psql_cred['psql']['password']

# Set up database connection
psql_connection = psycopg2.connect(dbname=dbname,
                   host= host,
                   port=port, 
                   user=user, 
                   password=password)


# Needed for connection
cursor = psql_connection.cursor()

# Print PostgreSQL Connection properties
print (psql_connection.get_dsn_parameters(),"\n")

{'user': 'cdog', 'dbname': 'twitter', 'host': '127.0.0.1', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



## Create a table in the database

A database called twitter was created through PgAdmin, but I wanted to practice creating tables with desired schemas in Python. At this point, I just assigned a Primary Key which will correspond to a counter variable running the stream for tweets. Right now, I'm not sure if this is the best way to go about this, especially if I were to index it. A clustered index around hashtags may prove to be better for my goal. At the moment, I'm just worried about getting going with Postgres & Python. I may make alterations down the line to optimize the retrieval of information. 

### Schema Creation

In [ ]:

# Schema for Postgres Table Information Collection
Table = """CREATE TABLE twitter_t
                (TID TEXT, 
                DATE_CREATED TEXT,
                TIMEZONE TEXT,
                NAME TEXT,
                USERDESC TEXT,
                RETWEET BOOLEAN,
                FULL_TEXT TEXT[],
                HASHTAG TEXT[],
                COORDINATE TEXT,
                GEO TEXT,
                PLACE TEXT,
                LOCATION TEXT,
                FAV_COUNT INT,
                QUOTE_COUNT INT,
                REPLY_COUNT INT,
                RETWEET_COUNT INT,
                STATUS_COUNT INT,
                SCREEN_NAME TEXT,
                FRIENDS_COUNT INT,
                LANG TEXT,
                FILTERLEVEL TEXT,
                SOURCE TEXT,
                SENSITIVE TEXT);"""



# Execute CREATE TABLE command
cursor.execute(Table)

# Commit the action to the database
psql_connection.commit()

# # Close the cursor
# psql_connection.close()
print("Table created successfully in PostgreSQL ")

## Twitter Script

This is the script I use to gather Twitter data and write it out to a record in my database. 

In [ ]:
psql_cred = jsc.json_config('config/psql_config.json')
dbname = psql_cred['psql']['dbname']
host = psql_cred['psql']['host']
port = psql_cred['psql']['port']
user = psql_cred['psql']['user']
password = psql_cred['psql']['password']

# Set up database connection
psql_connection = psycopg2.connect(dbname=dbname,
                   host= host,
                   port=port, 
                   user=user, 
                   password=password)

# Needed for connection
cursor = psql_connection.cursor()


print("Starting TwitterAPI")
api = TwitterAPI(consumer_key, consumer_secret, access_token_key, access_token_secret,)

# Look for keywords in tweets
keywords = "Warneford Hospital, warneford hospital, warnefordhospital, #warnefordhospital, john radcliffe hospital, John Radcliffe Hospital, #johnradcliffehospital"

while True:
    try:
        print("Setting up filter request")
        # We can add keyword filters (called 'track' term in Twitter)
        r = api.request('statuses/filter', {'track': keywords})
        


        print("Begin capturing tweets")
        for tweet in r:
     
            # Find hashtags
            hashtags = [i['text'] + " "  for i in tweet['entities']['hashtags']]

            # Filtering steps
            if tweet["retweeted"] == True:
                print("TRUE")

            # Retrieves the long description if there is one
            if 'extended_tweet' in tweet:

                text = tweet['extended_tweet']['full_text']

            else:
                text = tweet['text']

            # Formats coordinates if there are any
            if 'coordinates' in tweet and tweet['coordinates'] != None:
                coords = str(str((tweet['coordinates']["coordinates"][0]),str(tweet['coordinates']["coordinates"][1]))) # Add the tweet coordinates to the list
                print(tweet['coordinates'])
            else:
                coords = tweet['coordinates']# If the tweet does not have coordinates then skip it

            # Formats the place name if there is a long or short one
            if tweet['place'] != None:
                place = str(tweet['place']["full_name"]) # Add the tweet coordinates to the list

            else:
                place = tweet['place'] 

            retweeted = 'false'

            if text[:2] == 'RT':
                retweeted = 'true'
                text = text[2:]




            # To be included in the database
            tid = str(tweet['id'])
            date_created = tweet['created_at']
            timezone = tweet['user']['time_zone']
            user = tweet['user']['name']
            userdesc = tweet['user']['description']
            retweeted = retweeted
            full_text = [text]
            hashtags = hashtags
            coords = coords
            geo = tweet['geo']
            place = place
            fav_count = tweet['favorite_count']
            qoute_count = tweet['quote_count']
            reply_count = tweet['reply_count']
            retweet = tweet['retweet_count']
            status = tweet['user']['statuses_count']
            location = tweet['user']['location']
            screen_name = [tweet['user']['screen_name']]
            friends_count = tweet['user']['friends_count']
            lang = tweet['lang']
            filter_level = tweet['filter_level']
            source = tweet['source'].split('<')[-2].split('>')[-1]

            try:
                sensitive = tweet['possibly_sensitive']
            except:
                sensitive = 'None'


            # Execute query with insert statement of tweets
            cursor.execute(""" INSERT INTO twitter_t (TID, DATE_CREATED, TIMEZONE, NAME, USERDESC, RETWEET, FULL_TEXT, HASHTAG,COORDINATE,GEO,PLACE,
                                                             LOCATION, FAV_COUNT,QUOTE_COUNT,REPLY_COUNT,RETWEET_COUNT,STATUS_COUNT,
                                                             SCREEN_NAME,FRIENDS_COUNT, LANG, FILTERLEVEL, SOURCE, SENSITIVE) VALUES (%s, %s, %s, %s, %s,
                                                             %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""" , (tid, date_created, timezone, user, userdesc, retweeted, full_text, hashtags, coords, geo, place, 
                                                                                                         location, fav_count, qoute_count, reply_count, retweet, status, screen_name,
                                                                                                         friends_count, lang, filter_level, source, sensitive))
            # Commit database entry
            psql_connection.commit()

    
    except:
        print("Could capture tweet... \nERROR:", sys.exc_info()[0])
        pass
    
    time.sleep(15)

print("Done")




Starting TwitterAPI
Setting up filter request
Begin capturing tweets


Could capture tweet... 
ERROR: <class 'TwitterAPI.TwitterError.TwitterConnectionError'>
Setting up filter request
Begin capturing tweets


Could capture tweet... 
ERROR: <class 'TwitterAPI.TwitterError.TwitterConnectionError'>
Setting up filter request
Begin capturing tweets
